In [7]:
import numpy as np
import pandas as pd

from PIL import Image 
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import os
import re
import cv2

import skimage
from sklearn.base import BaseEstimator, TransformerMixin
from skimage.feature import hog

from scipy.sparse import csr_matrix
import h5py

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.base import BaseEstimator, TransformerMixin

import warnings
warnings.filterwarnings("ignore")

from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split



In [2]:
# Charger un tableau NumPy à partir du fichier HDF5
with h5py.File('data_0_comp.h5', 'r') as hf:
    X_train_img_id = hf['image_ids'][:]
    X_train_product_id = hf['product_ids'][:]
    X_train_hog = hf['X_train_hog'][:]
    X_train_hsv = hf['X_train_hsv'][:]
    

In [3]:
X_train_hog[:,0].shape

(10000,)

In [4]:
df_X_train_txt = pd.read_csv('text_lemm.csv', delimiter=',')

In [23]:
#on filtre sur les images qui ont bien été traitées
df_features = df_X_train_txt[df_X_train_txt['imageid'].isin(X_train_img_id)]
#df_features['X_train_hog'] = [row.tolist() for row in X_train_hog]
df_features['X_train_hog'] = X_train_hog[:,0]
X = df_features
y = df_features['prdtypecode']

In [82]:
#Initialisation of the TF-IDF vectorizer
vectorizer = TfidfVectorizer()
#Adding .values.astype('U') to avoid error np.nan is an invalid document, expected byte or unicode string.
X_train_tfidf = vectorizer.fit_transform(df_features['Text'].values.astype('U'))


In [69]:
# Séparer la variable explicative de la variable à prédire
X_tfidf, y_tfidf = df_X_train_txt['Text'], y

# Séparer le jeu de données en données d'entraînement et données test 
X_train_tfidf, X_test_tfidf, y_train_tfidf, y_test_tfidf = train_test_split(X_tfidf, y_tfidf, test_size=0.2, random_state = 30)

#Importer le package du TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer

# Créer un vectorisateur 
vec_tfidf = TfidfVectorizer()

# Mettre à jour la valeur de X_train_tfidf et X_test_tfidf
X_train_tfidf = vec_tfidf.fit_transform(X_train_tfidf)
X_test_tfidf = vec_tfidf.transform(X_test_tfidf)


In [70]:
# Créer un classificateur clf_tfidf et entraîner le modèle sur l'ensemble d'entraînement
clf_tfidf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0).fit(X_train_tfidf, y_train_tfidf)

# Calculer les prédictions 
y_pred_tfidf = clf_tfidf.predict(X_test_tfidf)


In [75]:
from sklearn.metrics import classification_report

# Calcul et affichage de classification_report
print( classification_report(y_test_tfidf, y_pred_tfidf) )

# Calcul et affichage de la matrice de confusion
conf_matrix_tfidf = pd.crosstab(y_test_tfidf, y_pred_tfidf, rownames=['Classe réelle'], colnames=['Classe prédite'])
conf_matrix_tfidf


              precision    recall  f1-score   support

          10       0.15      0.90      0.25        96
          40       0.53      0.17      0.25        54
          50       0.80      0.49      0.61        41
          60       0.00      0.00      0.00         7
        1140       0.00      0.00      0.00        75
        1160       0.88      0.75      0.81       173
        1180       0.86      0.63      0.73        19
        1280       0.49      0.53      0.51        73
        1281       0.48      0.51      0.49        45
        1300       0.53      0.22      0.31        36
        1301       1.00      0.14      0.25         7
        1302       0.80      0.31      0.44        13
        1320       0.65      0.38      0.48        73
        1560       0.71      0.56      0.63        18
        1920       0.33      0.10      0.15        10
        1940       0.72      0.67      0.69        27
        2060       0.60      0.30      0.40        20
        2220       0.33    

Classe prédite,10,40,50,1160,1180,1280,1281,1300,1301,1302,...,2060,2220,2280,2403,2462,2522,2582,2583,2585,2705
Classe réelle,,,,,,,,,,,,,,,,,,,,,
10,86,0,0,2,1,0,0,0,0,0,...,0,0,5,1,0,0,0,0,0,1
40,38,9,0,0,0,0,3,0,0,0,...,0,0,0,2,0,1,0,0,0,1
50,11,1,20,1,0,0,1,1,0,0,...,1,0,0,2,1,0,0,0,0,1
60,2,0,1,0,0,0,2,0,0,0,...,0,0,0,0,1,0,0,0,0,1
1140,29,1,0,1,0,34,1,0,0,0,...,0,0,5,2,0,0,0,0,0,0
1160,38,0,0,129,0,0,0,1,0,0,...,0,0,4,0,0,1,0,0,0,0
1180,5,0,0,0,12,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1280,20,2,0,0,0,39,2,0,0,0,...,1,0,0,0,0,1,1,1,0,0
1281,17,0,0,1,0,1,23,0,0,0,...,0,0,0,0,1,2,0,0,0,0


In [76]:
X_train_tfidf

<8000x30857 sparse matrix of type '<class 'numpy.float64'>'
	with 156354 stored elements in Compressed Sparse Row format>

In [83]:
X = X_train_tfidf  # Remplacez-le par le vecteur TF-IDF ou concaténez avec X_image si nécessaire
X_train, X_test, X_image_train, X_image_test, y_train, y_test = train_test_split(X, X_train_hog, y, test_size=0.2, random_state=42)


In [84]:
# Créer un pipeline pour les vecteurs TF-IDF
pipeline_tfidf = Pipeline([
    ('tfidf', TfidfVectorizer()),
])

# Créer une classe de transformateur personnalisé pour X_image
class ArrayTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return X
    
# Créer une instance de la classe de transformateur personnalisé pour X_image
transformateur_X_image = ArrayTransformer()

# Créer un FeatureUnion pour fusionner les deux ensembles de fonctionnalités
union = FeatureUnion([
    ('pipeline_tfidf', pipeline_tfidf),
    ('transformateur_X_image', transformateur_X_image)
])

In [86]:
# Créer un pipeline complet avec GradientBoostingClassifier
pipeline = Pipeline([
    ('union', union),
    ('classifier', GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0))
])

In [150]:
X_train

,productid,imageid,prdtypecode,Text,String,X_train_hog
78596,570613273,1027214826,2280,"['figaro', 'economie', 'ndeg', 'france', 'alle...",figaro economie ndeg france allemagne locomoti...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
13415,436061491,1008137688,2280,"['ndeg', 'collection']",ndeg collection,"[0.3644168449413844, 0.001254353959454423, 0.0..."
14317,1035122315,1053586721,1160,"['yu-gi-oh', 'dpc', '-fr', 'arachnee', 'souter...",yu-gi-oh dpc -fr arachnee souterraine super ra...,"[0.028867260594846813, 0.014331951688977551, 0..."
51876,1413003884,1078972014,2583,"['bache', 'bulles', 'bordee', 'ubbink', 'bache...",bache bulles bordee ubbink bache bulle piscine...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
56725,570628331,1027257176,2280,"['mystere', 'magazine', 'ndeg', 'maitres', 'po...",mystere magazine ndeg maitres policier jeanne ...,"[0.0081529289634611, 0.0042981760647280804, 0...."
...,...,...,...,...,...,...
49037,1030803722,1053394296,2403,"['alexandre', 'duma', 'crime', 'celebres', 'to...",alexandre duma crime celebres tome,"[0.1748161557459988, 0.011835792771550887, 0.0..."
44608,435096821,1007592393,2280,"['bulletin', 'quotidien', 'presse', 'etrangere...",bulletin quotidien presse etrangere ndeg quest...,"[0.20915827621051183, 0.14955281866696846, 0.1..."
46039,435076998,1007659157,2280,"['canard', 'enchaine', 'ndeg', 'ump', 'craint'...",canard enchaine ndeg ump craint branlee electi...,"[0.42482526347212884, 0.1626376779960767, 0.20..."
7416,387910861,1022644613,1300,"['citroen', 'dinky', 'toy', 'ref', 'reedition'...",citroen dinky toy ref reedition atlas norev ou...,"[0.2901741815916107, 0.008697537943322911, 0.0..."


In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [26]:
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.compose import make_column_transformer
import numpy as np

# Transformer pour les données textuelles
text_transformer = Pipeline([
    ('tfidf', TfidfVectorizer()),
])

# Créer une instance de l'identité transformateur
image_transformer = IdentityTransformer()

# Transformer pour les données d'images (ajuster en fonction de votre méthode de traitement d'image)
#image_transformer = Pipeline([
#    ('hog', HogTransformer()) # Mise à l'échelle des données
#])


In [27]:
text_transformer.fit_transform(X_train['Text'])


<8000x30509 sparse matrix of type '<class 'numpy.float64'>'
	with 153704 stored elements in Compressed Sparse Row format>

In [28]:
image_transformer.fit_transform(X_train['X_train_hog'])


78596    0.000000
13415    0.364417
14317    0.028867
51876    0.000000
56725    0.008153
           ...   
49037    0.174816
44608    0.209158
46039    0.424825
7416     0.290174
61981    0.000000
Name: X_train_hog, Length: 8000, dtype: float64

In [29]:

union = FeatureUnion([
    ('pipeline_tfidf', text_transformer),
    ('transformateur_X_image', IdentityTransformer)
])

preprocessor = make_column_transformer((text_transformer, ['Text']),
                                     (image_transformer, ['X_train_hog']))


copie = X.copy()

preprocessor.fit_transform(copie)



ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 0, the array at index 0 has size 1 and the array at index 1 has size 10000

In [ ]:

# Classifieur
classifier = SVC()

# Créer le pipeline complet
pipeline = Pipeline([
    ('features', combined_features),
    ('classifier', classifier),
])

# Entraîner le pipeline
pipeline.fit([('text_column_name', X_train), ('image_column_name', X_image_train)], y_train)

# Prédiction
predictions = pipeline.predict({'text_column_name': X_test, 'image_column_name': X_image_test})



In [120]:
X_train.shape

(8000,)

In [121]:
X_image_train.shape

(8000, 8100)

In [13]:
# Transformer identité pour les données d'images
class IdentityTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return X
        

In [10]:
class HogTransformer(BaseEstimator, TransformerMixin):
    """
    Expects an array of 2d arrays (1 channel images)
    Calculates hog features for each img
    """
 
    def __init__(self, y=None, orientations=9,
                 pixels_per_cell=(14,14),
                 cells_per_block=(2,2), block_norm='L2-Hys'):
        self.y = y
        self.orientations = orientations
        self.pixels_per_cell = pixels_per_cell
        self.cells_per_block = cells_per_block
        self.block_norm = block_norm
 
    def fit(self, X, y=None):
        return self
 
    def transform(self, X, y=None):
 
        def local_hog(X):
            return hog(X,
                       orientations=self.orientations,
                       pixels_per_cell=self.pixels_per_cell,
                       cells_per_block=self.cells_per_block,
                       block_norm=self.block_norm)
 
        try: # parallel
            return np.array([local_hog(img) for img in X])
        except:
            return np.array([local_hog(img) for img in X])
            